##

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import torch

import torchvision
from torchvision import datasets, transforms

from collections import Counter
from torch.utils.data import random_split

In [2]:
seed = 10
torch.manual_seed(seed)

category_index = 8
n_val = 5000

data_path = '/cifar-10-batches-py'

Function for loading the Cifar10 dataset.

The method will have to be run twice.
After running the method for the first time we get create a normalizer from the std and mean of the images. The method is then ran for a second time with the normalizer as the preprocessor.

Loading the CIFAR-10 dataset as tensors.

In [3]:
transformed_cifar10_train_val = datasets.CIFAR10(
    data_path,
    train=True,
    download=True,
    transform = transforms.ToTensor()
)

Files already downloaded and verified


Stacking the set of images into a single tensor. We then create a normalizer for the dataset around the mean and standard deviation of the 3 dimensions (height, width channel (color)).

In [4]:
imgs = torch.stack([img for img, _ in transformed_cifar10_train_val])

normalizer = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean = imgs.mean(dim=(0, 2, 3)), std = imgs.std(dim=(0, 2, 3)))])

Loading the dataset as tensors for training+validation and testing. This time we apply the composition of transforms.

In [5]:
normalized_cifar10_train_val = datasets.CIFAR10(
    data_path,
    train=True,
    download=False,
    transform = normalizer
)


transformed_cifar10_test = datasets.CIFAR10(
    data_path,
    train=False,
    download=False,
    transform = normalizer
)

As this is a binary classification problem where we only want to identify whether an image is a ship or not, we can set the labels that are "ship" to true. We set all other labels to false.

In [11]:
train_labels = np.array([label for _, label in transformed_cifar10_train_val])
train_labels = np.array(train_labels == category_index).astype(int)

test_labels = np.array([label for _, label in transformed_cifar10_test])
test_labels = np.array(test_labels == category_index).astype(int)

Splitting the training and validation set randomly.

In [7]:
n_train = len(transformed_cifar10_train_val)-n_val

transformed_cifar10_train_split, transformed_cifar10_val_split = random_split(
    transformed_cifar10_train_val,
    [n_train, n_val],

    generator=torch.Generator().manual_seed(seed)
)

print("Size of the train dataset:        ", len(transformed_cifar10_train_split))
print("Size of the validation dataset:   ", len(transformed_cifar10_val_split))
print("Size of the test dataset:         ", len(transformed_cifar10_test))

Counter([label for _, label in transformed_cifar10_train_split])

Size of the train dataset:         45000
Size of the validation dataset:    5000
Size of the test dataset:          10000


Counter({8: 4486,
         7: 4486,
         1: 4512,
         6: 4510,
         3: 4516,
         5: 4487,
         4: 4499,
         2: 4483,
         0: 4518,
         9: 4503})

Choosing a pre-trained CNN model: we chose ResNet18, which is not trained on Cifar-10.

In [8]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim

Loading pre-trained ResNet18 model and modifying the last layer. We are doing binary classification, so we think we only need one node in the final layer.

In [9]:
model = models.resnet18(pretrained=True)

num_features = model.fc.in_features
model.fc = nn.Linear(num_features,1)

c:\ProgramData\Anaconda3\envs\inf265\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\inf265\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


We use Binary Cross Entropy as it should be suitable for binary classification problems (add reasoning and explanation). We use nn.BCEWithLogitsLoss() as it combines the sigmoid activation function and the BCE into a single class.

The optimizer we use is Adam, and we will begin with a learning rate of 0.001, just because it is a commonly used learning rate.

In [10]:
loss_function = nn.BCEWithLogitsLoss()


optimizer = optim.adam(model.parameters(),  lr=0.001)

AttributeError: module 'torch.optim' has no attribute 'adam'